<a href="https://colab.research.google.com/github/PurvaChiniya/LearnIT-Girl-Project/blob/master/en_to_french.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import copy
import pickle
def load_data(path):
  input_file=os.path.join(path)
  with open(input_file, 'r',  encoding = "ISO-8859-1") as f:
        data = f.read()
  return data
source_path="en.txt"
target_path="french.txt"
source_text=load_data(source_path)
target_text=load_data(target_path)
CODES={"<PAD>":0,"<EOS>":1,"<UNK>":2,"<GO>":3}
def convert_to_tables(text):
  vocab=set(text.split())
  vocab_to_int=copy.copy(CODES)
  for v_i,v in enumerate (vocab,len(CODES)):
    vocab_to_int[v]=v_i
  int_to_vocab={v_i:v for v,v_i in vocab_to_int.items()}
  return vocab_to_int,int_to_vocab

FileNotFoundError: ignored

In [0]:
def text_to_ids(source_text,target_text,source_vocab_to_int,target_vocab_to_int):
  source_text_id=[]
  target_text_id=[]
  source_sentences=source_text.split("\n")
  target_sentences=target_text.split("\n")
  max_source_sentence_length=max([len(sentence.split(" ")) for sentence in source_sentences])
  max_target_sentence_length=max([len(sentence.split(" ")) for sentence in target_sentences])
  for i in range(len(source_sentences)):
    source_sentence=source_sentences[i]
    target_sentence=target_sentences[i]
    source_tokens=source_sentence.split(" ")
    target_tokens=target_sentence.split(" ")
    source_token_id=[]
    target_token_id=[]
    for index,token in enumerate(source_tokens):
      if(token !=""):
        source_token_id.append(source_vocab_to_int[token])
    for index,token in enumerate(target_tokens):
      if (token !=""):
        target_token_id.append(target_vocab_to_int[token])
    target_token_id.append(target_vocab_to_int["<EOS>"])
    source_token_id.append(source_token_id)
    target_token_id.append(target_token_id)
  return source_text_id,target_text_id    

In [0]:
def preprocess_data_and_save_data(source_path,target_path,text_to_ids):
  source_text=load_data(source_path)
  target_text=load_data(target_path)
  source_text=source_text.lower()
  target_text=target_text.lower()
  source_vocab_to_int,source_int_to_vocab=convert_to_tables(source_text)
  target_vocab_to_int,target_int_to_vocab=convert_to_tables(target_text)
  source_text,target_text=text_to_ids(source_text,target_text,source_vocab_to_int,target_vocab_to_int)
  pickle.dump(((source_text,target_text),(source_vocab_to_int,target_vocab_to_int),(source_int_to_vocab,target_vocab_to_int)),open("preprocess.p","wb"))
  

In [0]:
preprocess_data_and_save_data(source_path,target_path,text_to_ids)

CHECKPOINT

In [0]:
def load_preprocess():
    with open('preprocess.p', mode='rb') as in_file:
        return pickle.load(in_file)
import numpy as np
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()      

In [0]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.12.0
Default GPU Device: /device:GPU:0


NUERAL NETWORK

In [0]:
def inputs():
  inputs=tf.placeholder(tf.int32,[None,None],name="inputs")
  targets=tf.placeholder(tf.int32,[None,None],name="targets")
  target_sequence_length=tf.placeholder(tf.int32,[None],name="target_sequence_length")
  max_target_length=tf.reduce_max(target_sequence_length)
  return inputs,targets,target_sequence_length
  

In [0]:
def hyperparam_inputs():
  lr_rate=tf.placeholder(tf.float32,name="lr_rate")
  keep_prob=tf.placeholder(tf.float32,name="keep_prob")
  return lr_rate,keep_prob
  

In [0]:
def process_decoder_input(target_data,target_vocab_to_int,batch_size):
  go_id=target_vocab_to_int["<GO>"]
  after_slice=tf.strided_slice(target_data,[0,0],[batch_size,-1],[1,1])
  after_concat=tf.concat([tf.fill([batch_size,1],go_id),after_slice],1)
  return after_concat

In [0]:

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_vocab_size, 
                   encoding_embedding_size):
    """
    :return: tuple (RNN output, RNN state)
    """
    embed = tf.contrib.layers.embed_sequence(rnn_inputs, 
                                             vocab_size=source_vocab_size, 
                                             embed_dim=encoding_embedding_size)
    
    stacked_cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(rnn_size), keep_prob) for _ in range(num_layers)])
    
    outputs, state = tf.nn.dynamic_rnn(stacked_cells, 
                                       embed, 
                                       dtype=tf.float32)
    return outputs, state

In [0]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a training process in decoding layer 
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    # for only input layer
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, 
                                               target_sequence_length)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              encoder_state, 
                                              output_layer)

    # unrolling the decoder layer
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_summary_length)
    return outputs

In [0]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a inference process in decoding layer 
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                      tf.fill([batch_size], start_of_sequence_id), 
                                                      end_of_sequence_id)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              encoder_state, 
                                              output_layer)
    
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_target_sequence_length)
    return outputs

In [0]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    target_vocab_size = len(target_vocab_to_int)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(rnn_size) for _ in range(num_layers)])
    
    with tf.variable_scope("decode"):
        output_layer = tf.layers.Dense(target_vocab_size)
        train_output = decoding_layer_train(encoder_state, 
                                            cells, 
                                            dec_embed_input, 
                                            target_sequence_length, 
                                            max_target_sequence_length, 
                                            output_layer, 
                                            keep_prob)

    with tf.variable_scope("decode", reuse=True):
        infer_output = decoding_layer_infer(encoder_state, 
                                            cells, 
                                            dec_embeddings, 
                                            target_vocab_to_int['<GO>'], 
                                            target_vocab_to_int['<EOS>'], 
                                            max_target_sequence_length, 
                                            target_vocab_size, 
                                            output_layer,
                                            batch_size,
                                            keep_prob)

    return (train_output, infer_output)

In [0]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence model
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    enc_outputs, enc_states = encoding_layer(input_data, 
                                             rnn_size, 
                                             num_layers, 
                                             keep_prob, 
                                             source_vocab_size, 
                                             enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, 
                                      target_vocab_to_int, 
                                      batch_size)
    
    train_output, infer_output = decoding_layer(dec_input,
                                               enc_states, 
                                               target_sequence_length, 
                                               max_target_sentence_length,
                                               rnn_size,
                                              num_layers,
                                              target_vocab_to_int,
                                              target_vocab_size,
                                              batch_size,
                                              keep_prob,
                                              dec_embedding_size)
    
    return train_output, infer_output


In [0]:
display_step = 300

epochs = 13
batch_size = 128

rnn_size = 128
num_layers = 3

encoding_embedding_size = 200
decoding_embedding_size = 200

learning_rate = 0.001
keep_probability = 0.5

In [0]:
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, target_sequence_length, max_target_sequence_length = enc_dec_model_inputs()
    lr, keep_prob = hyperparam_inputs()
    
    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)
    
    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    # https://www.tensorflow.org/api_docs/python/tf/sequence_mask
    # - Returns a mask tensor representing the first N positions of each cell.
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function - weighted softmax cross entropy
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)